<a href="https://colab.research.google.com/github/hamel-amir/Projet_Big_Data/blob/main/Projet_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Big Data : Apache Spark

Réalisé par:

* Amir  Hamel
* Noor Khalal
* Zahra Alliche

## Objectifs du TP
Ce TP consiste à regrouper des documents textuels tels que les documents qui
partagent le même thème se retrouvent dans le même groupe, et les documents qui
portent sur des sujets très différents se trouvent dans des groupes différents.

## 2 Mise en place de l'environnement de travail 

### 2.2 Installation de Spark

**Instalation de la bibliotheque Java**

In [1]:
! apt-get install openjdk-8-jdk-headless -qq > /dev/null

**Instalation de Spark**

In [2]:
! wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz


In [3]:
! tar xf spark-3.3.2-bin-hadoop3.tgz

**Instalation de PySpark**

In [4]:
# instalation de pyspark
! pip install -q findspark
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2.3/4 Définir la vatiable d'environnement et créer l'objet SparkContext 

In [5]:
import os
# definir deux variables d'environement
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"  # l'endroit de java
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3" # l'endroit de spark
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.5 pyspark-shell' 

import findspark
findspark.init("spark-3.3.2-bin-hadoop3")

from pyspark import SparkContext, SparkConf

# lancement du spark en local (en une seule machine qui est la machine virtuelle de colab dans ce cas) avec 4 processus (workers node)
configuration = SparkConf().setAppName("name").setMaster("local[4]").set('spark.jars.packages', 'org.apache.spark:spark-avro_2.11:2.4.5')
# name: c'est le nom qu'on donne a notre app (code)
sc = SparkContext(conf=configuration) # l'objet 

In [6]:
sc

<SparkContext master=local[4] appName=name>

**Création de l'objet sparkSession**

In [7]:
# L'objet sparkSession créé pour utiliser l'API Spark SQL
from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=configuration).getOrCreate()

## 3 Données

### 3.1 Téléchargement des données

In [8]:
# Téléchargement du dossier des documents
! wget -q http://qwone.com/~jason/20Newsgroups/20news-19997.tar.gz

### 3.2 Decompression des données

In [9]:
# décompresser le dossier 
! tar xf /content/20news-19997.tar.gz

### 3.3 Chargement des données dans deux variables de type RDD



In [10]:
rdd1 = sc.wholeTextFiles("/content/20_newsgroups/alt.atheism")
rdd2 = sc.wholeTextFiles("/content/20_newsgroups/rec.sport.baseball")

In [11]:
x=rdd2.take(2)

###3.4 Séparation de l’entête

In [12]:
def separateur(x):
  l=x[1].split("\n\n")
  return (x[0],l)

rdd1=rdd1.map(separateur)
rdd2=rdd2.map(separateur)

In [13]:
# tester le premier élément
test=rdd1.take(1)
test

[('file:/content/20_newsgroups/alt.atheism/53791',
  ['Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53791 talk.religion.misc:84109 talk.origins:41079\nNewsgroups: alt.atheism,talk.religion.misc,talk.origins\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!zaphod.mps.ohio-state.edu!swrinde!cs.utexas.edu!utnut!torn!nott!cunews!rjg\nFrom: rjg@doe.carleton.ca (Richard Griffith)\nSubject: Re: Burden of Proof\nMessage-ID: <rjg.735427215@wesley>\nSender: news@cunews.carleton.ca (News Administrator)\nOrganization: Dept. of Electronics, Carleton University\nReferences: <C5t5sF.8oz@mentor.cc.purdue.edu> <1r4b59$7hg@aurora.engr.LaTech.edu>\nDate: Wed, 21 Apr 1993 21:20:15 GMT\nLines: 23',
   'In <1r4b59$7hg@aurora.engr.LaTech.edu> ray@engr.LaTech.edu (Bill Ray) writes:',
   '>If I make a statement, "That God exists, loves me, etc." but in no way\n>insist that you believe it, does that place a 

On voit ici qu'un document est représenté sous la forme d'un tuple `(Path, List)`,

 ou `List = [entête, contenu du document ]`

### 3.5 Extraction des champs de l’entête

Création d'une fonction d'extraction des éléments de l’entête qui
prend en parametre l’entête d'un document et
retourne un dictionnaire (Intitulé_Champs, Contenu_Champs)

In [14]:


def extraction_informations_entete(entete):
  #chaque ligne représente un nouveau champs
  l=entete.split("\n")
  d=dict()

  for x in l:
    #l'intitulé du champs est séparé de son contenu par ':'
    z=x.split(":")
    #quelques lignes ne contiennent pas de ':'
    if len(z)==2:
       d[z[0]]=z[1]

  return d

#### Premier exemple d'extraction

Extraire des informations du premier document du rdd1 (alt.atheism)

In [15]:

# par l'action first
doc1=rdd1.first()
entete=doc1[1][0]

# Dictionnaire des informations de l'entete
dict1_info=extraction_informations_entete(entete)
dict1_info

{'Newsgroups': ' alt.atheism,talk.religion.misc,talk.origins',
 'Path': ' cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!zaphod.mps.ohio-state.edu!swrinde!cs.utexas.edu!utnut!torn!nott!cunews!rjg',
 'From': ' rjg@doe.carleton.ca (Richard Griffith)',
 'Message-ID': ' <rjg.735427215@wesley>',
 'Sender': ' news@cunews.carleton.ca (News Administrator)',
 'Organization': ' Dept. of Electronics, Carleton University',
 'References': ' <C5t5sF.8oz@mentor.cc.purdue.edu> <1r4b59$7hg@aurora.engr.LaTech.edu>',
 'Lines': ' 23'}

On va extraire NewsGroups et Organisation de l'élément dict1_info

In [16]:
# selectionner Organization et Newsgroups
print(dict1_info['Newsgroups'])
print(dict1_info['Organization'])

 alt.atheism,talk.religion.misc,talk.origins
 Dept. of Electronics, Carleton University


#### Deuxieme exemple d'extraction

In [17]:
# par l'action take
doc2=rdd2.take(2)
entete2=doc2[1][1][0]
# Dictionnaire des informations de l'entete
dict2_info=extraction_informations_entete(entete2)
dict2_info

{'Newsgroups': ' rec.sport.baseball',
 'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!usenet.ins.cwru.edu!wsu-cs!vela.acs.oakland.edu!cs.uiuc.edu!cs.uiuc.edu!steph',
 'From': ' steph@cs.uiuc.edu (Dale Stephenson)',
 'Subject': ' Defensive Averages 1988-1992, Third Base',
 'Message-ID': ' <C5JJrJ.EM3@cs.uiuc.edu>',
 'Summary': ' career defensive averages at third',
 'Organization': ' University of Illinois, Dept. of Comp. Sci., Urbana, IL',
 'Lines': ' 68'}

In [18]:
# selectionner Organization et Newsgroups
print(dict2_info['Newsgroups'])
print(dict2_info['Organization'])

 rec.sport.baseball
 University of Illinois, Dept. of Comp. Sci., Urbana, IL


### 3.6 Fusion des deux new RDD

Ici, nous allons créer une nouvelle fonction de séparation de document qui va créer un élément de RDD de la forme

`(Path, [Dictionnaire de l'entete , le contenu du document])`

Ceci rendra la transformation des élément en type pyspark.sql.Row plus facile durant le choix des colonnes car certains champs de l'entete n'éxistent pas dans tous les documents, le test d'éxistance sera plus facile grace au dictionnaire.

In [19]:
# cette fonction prend en entrée un élement du rdd (doc)
# et retourne un tuple contenant ( path, [dictionnaire de l'entete , le contenu du document])
def structuration_document(doc):
  if  doc is not None:
    l= doc[1][0].split("\n")
    d=dict()
    if len(l)!=0:
      for x in l:
        z=x.split(":",1)
        if len(z)==2:
          d[z[0]]=z[1]
        if len(z)<2:
           pass
        

      return (doc[0],[d,doc[1][1]])

In [20]:
new_rdd1=rdd1.map(structuration_document)


In [21]:
# affichage du resultat de new_rdd1
new_rdd1.take(2)

[('file:/content/20_newsgroups/alt.atheism/53791',
  [{'Xref': ' cantaloupe.srv.cs.cmu.edu alt.atheism:53791 talk.religion.misc:84109 talk.origins:41079',
    'Newsgroups': ' alt.atheism,talk.religion.misc,talk.origins',
    'Path': ' cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!zaphod.mps.ohio-state.edu!swrinde!cs.utexas.edu!utnut!torn!nott!cunews!rjg',
    'From': ' rjg@doe.carleton.ca (Richard Griffith)',
    'Subject': ' Re: Burden of Proof',
    'Message-ID': ' <rjg.735427215@wesley>',
    'Sender': ' news@cunews.carleton.ca (News Administrator)',
    'Organization': ' Dept. of Electronics, Carleton University',
    'References': ' <C5t5sF.8oz@mentor.cc.purdue.edu> <1r4b59$7hg@aurora.engr.LaTech.edu>',
    'Date': ' Wed, 21 Apr 1993 21:20:15 GMT',
    'Lines': ' 23'},
   'In <1r4b59$7hg@aurora.engr.LaTech.edu> ray@engr.LaTech.edu (Bill Ray) writes:']),
 ('file:/content/20_newsgro

In [22]:
# en appliquant la meme fonction sur le rdd2
new_rdd2=rdd2.map(structuration_document)

In [23]:
new_rdd2.take(2)

[('file:/content/20_newsgroups/rec.sport.baseball/104985',
  [{'Newsgroups': ' rec.sport.baseball',
    'Path': ' cantaloupe.srv.cs.cmu.edu!rochester!udel!gatech!howland.reston.ans.net!zaphod.mps.ohio-state.edu!magnus.acs.ohio-state.edu!dmoney',
    'From': ' dmoney@magnus.acs.ohio-state.edu (Dean R Money)',
    'Subject': ' The Braves will come around...',
    'Message-ID': ' <1993Apr23.193027.26515@magnus.acs.ohio-state.edu>',
    'Sender': ' news@magnus.acs.ohio-state.edu',
    'Nntp-Posting-Host': ' bottom.magnus.acs.ohio-state.edu',
    'Organization': ' The Ohio State University',
    'Date': ' Fri, 23 Apr 1993 19:30:27 GMT',
    'Lines': ' 28'},
   'To all the Braves doubters:']),
 ('file:/content/20_newsgroups/rec.sport.baseball/104504',
  [{'Newsgroups': ' rec.sport.baseball',
    'Path': ' cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!usenet.ins.cwru.edu!wsu-cs!vela.acs.oakland.edu!cs.uiuc.edu!cs.uiuc.edu!steph',
    'From': ' steph@cs.uiu

en faisant l'union des deux RDD

In [24]:
# Fusionner les deux RDD (union)
fusion=new_rdd1.union(new_rdd2)

In [25]:
fusion.take(2)

[('file:/content/20_newsgroups/alt.atheism/53791',
  [{'Xref': ' cantaloupe.srv.cs.cmu.edu alt.atheism:53791 talk.religion.misc:84109 talk.origins:41079',
    'Newsgroups': ' alt.atheism,talk.religion.misc,talk.origins',
    'Path': ' cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!howland.reston.ans.net!zaphod.mps.ohio-state.edu!swrinde!cs.utexas.edu!utnut!torn!nott!cunews!rjg',
    'From': ' rjg@doe.carleton.ca (Richard Griffith)',
    'Subject': ' Re: Burden of Proof',
    'Message-ID': ' <rjg.735427215@wesley>',
    'Sender': ' news@cunews.carleton.ca (News Administrator)',
    'Organization': ' Dept. of Electronics, Carleton University',
    'References': ' <C5t5sF.8oz@mentor.cc.purdue.edu> <1r4b59$7hg@aurora.engr.LaTech.edu>',
    'Date': ' Wed, 21 Apr 1993 21:20:15 GMT',
    'Lines': ' 23'},
   'In <1r4b59$7hg@aurora.engr.LaTech.edu> ray@engr.LaTech.edu (Bill Ray) writes:']),
 ('file:/content/20_newsgro

### 3.7 Transformation le nouveau RDD obtenu pour que chaque élément soit de type pyspark.sql.Row

In [26]:
from pyspark.sql import Row

# cette fonction ToRow est utilisée pour manipuler la table sql et faire des requetes.
def ToRow(x):
  entete=x[1][0]
  
  #Certains éléments (comme Subject ou Date) n'éxistent pas dans tous les documents
  #C'est pour cela que la création du dictionnaire avec la fonction structuration_document() nous aide dans les tests d'éxistance
  row = Row(Path=x[0], Newsgroups=entete['Newsgroups'] if 'Newsgroups' in entete.keys() else None,
            Organization=entete['Organization'] if 'Organization' in entete.keys() else None,
            Subject=entete['Subject'] if 'Subject' in entete.keys() else None,
            Date=entete['Date'] if 'Date' in entete.keys() else None,
            Lines=entete['Lines'] if 'Lines' in entete.keys() else None,
            From=entete['From'] if 'From' in entete.keys() else None,

            Contenu=x[1][1])
  
  return row

rddF=fusion.map(ToRow)


In [27]:
rddF.take(2)

[Row(Path='file:/content/20_newsgroups/alt.atheism/53791', Newsgroups=' alt.atheism,talk.religion.misc,talk.origins', Organization=' Dept. of Electronics, Carleton University', Subject=' Re: Burden of Proof', Date=' Wed, 21 Apr 1993 21:20:15 GMT', Lines=' 23', From=' rjg@doe.carleton.ca (Richard Griffith)', Contenu='In <1r4b59$7hg@aurora.engr.LaTech.edu> ray@engr.LaTech.edu (Bill Ray) writes:'),
 Row(Path='file:/content/20_newsgroups/alt.atheism/54220', Newsgroups=' alt.atheism', Organization=' Case Western Reserve University', Subject=' Re: free moral agency', Date=' Thu, 22 Apr 1993 23:48:54 GMT', Lines=' 29', From=' kmr4@po.CWRU.edu (Keith M. Ryan)', Contenu='In article <C5uxJ9.pJ@darkside.osrhe.uoknor.edu> bil@okcforum.osrhe.edu (Bill Conner) writes:')]

### 3.8 Création d'un objet de type DataFrame à partir du RDD fusionné

In [66]:
#Create dataframe spark
df=spark.createDataFrame(rddF)
df.printSchema()

root
 |-- Path: string (nullable = true)
 |-- Newsgroups: string (nullable = true)
 |-- Organization: string (nullable = true)
 |-- Subject: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Lines: string (nullable = true)
 |-- From: string (nullable = true)
 |-- Contenu: string (nullable = true)



In [67]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|                Path|          Newsgroups|        Organization|             Subject|                Date|Lines|                From|             Contenu|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
|file:/content/20_...| alt.atheism,talk...| Dept. of Electro...| Re: Burden of Proof| Wed, 21 Apr 1993...|   23| rjg@doe.carleton...|In <1r4b59$7hg@au...|
|file:/content/20_...|         alt.atheism| Case Western Res...| Re: free moral a...| Thu, 22 Apr 1993...|   29| kmr4@po.CWRU.edu...|In article <C5uxJ...|
|file:/content/20_...|         alt.atheism| California Insti...| Re: >>>>>>Pompou...| 2 Apr 93 21:01:4...|   14| keith@cco.caltec...|livesey@solntze.w...|
|file:/content/20_...|         alt.atheism| Decision Support...| Re: Y

In [68]:
df.count()

2000

### 3.9 Sauvegarde la DataFrame au format Avro

❌ **Remarque:** Nous n'avons pas trouvé une solution de sauvegarde aux formats Avro et Parquet sans passer par pandas. Cependant, cette solution n'est pas optimale car toutes les données seront envoyées au Driver node.



In [29]:
!pip install pandavro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import numpy as np
import pandas as pd
import pandavro as pdx
from pyspark.sql.avro.functions import from_avro, to_avro
filename = "df.avro"

# on est passé par le dataframe pandas mais ça envoie tout au driver
df2=df.toPandas()
pdx.to_avro(filename, df2)


### 3.10 Sauvegarder la DataFrame au format Parquet

In [32]:
# sauvgarder le dataframe au format parquet
import pyarrow as pa
import pyarrow.parquet as pq
table = pa.Table.from_pandas(df2)
pq.write_table(table, 'df.parquet')

## 4 Analyse descriptive API SPARK SQL

### 4.1 Vérification de deux catégories différentes de documents

In [34]:
# le nombre de documents par Newsgroups
df.groupBy("Newsgroups").count().show()

+--------------------+-----+
|          Newsgroups|count|
+--------------------+-----+
| talk.religion.mi...|    2|
| sci.skeptic,alt....|    7|
| alt.atheism,soc....|    1|
| talk.abortion,al...|    1|
| alt.atheism,talk...|    3|
| alt.atheism,rec....|    3|
| alt.atheism,talk...|   92|
| rec.scouting,soc...|    1|
| soc.culture.arab...|    2|
| alt.atheism,talk...|    5|
| alt.atheism,alt....|    2|
| alt.atheism,soc....|    8|
|         alt.atheism|  734|
| alt.atheism,alt....|    7|
| talk.religion.mi...|    4|
| alt.atheism,talk...|   20|
| alt.slack,talk.r...|    1|
| talk.religion.mi...|    4|
| talk.abortion,al...|   94|
| alt.atheism,soc....|    2|
+--------------------+-----+
only showing top 20 rows



On remarque que le champ Newsgroups peut contenir des éléments en plus des deux catégories principales "alt.atheism" ou "rec.sport.baseball".

Pour vérifier qu'on a bien ces deux catégories principales différentes, nous allons créer deux vues: 

* Une vue `sport` qui contient toutes les lignes où `rec.sport.baseball` apparait.
* Une vue `religion` qui contient toutes les lignes où `alt.atheism` apparait.

Puis nous allons compter le nombre d'éléments dans chaque liste, en plus de voir si l'intersectiondes deux vues est vide ou pas.

Si l'intersection est vide, et que la somme des éléments des deux vues est égale au nombre total des éléments de la fusion, cela veut dire que les documents sont bien separés par deux catégories différentes  "alt.atheism" et "rec.sport.baseball".

In [35]:
# Mettre le dataframe dans une table sql (Document)

from pandas._libs.hashtable import value_count
df.createOrReplaceTempView("Document")

Si les vues sont déja créées, executer ces deux lignes:

In [36]:
#Si les vues sont déja créées
#spark.sql("DROP VIEW sport")
#spark.sql("DROP VIEW religion")

In [37]:
# Création d'une vue (sport) contennat tous les document de catégorie rec.sport.baseball
sqlDF = spark.sql("CREATE TEMP VIEW sport AS SELECT Newsgroups FROM Document WHERE Newsgroups LIKE '%rec.sport.baseball%'")

In [38]:
# Création d'une vue (religion) contennat tous les document de catégorie alt.atheism
sqlDF = spark.sql("CREATE TEMP VIEW religion AS SELECT Newsgroups FROM Document WHERE Newsgroups LIKE '%alt.atheism%'")

In [39]:
# affichage la vue Sport
sqlDF2 = spark.sql("SELECT * FROM sport")
sqlDF2.show()

+--------------------+
|          Newsgroups|
+--------------------+
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
| rec.sport.baseba...|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
|  rec.sport.baseball|
+--------------------+
only showing top 20 rows



In [40]:
# Affichage la vue religion
sqlDF3 = spark.sql("SELECT * FROM religion")
sqlDF3.show()

+--------------------+
|          Newsgroups|
+--------------------+
| alt.atheism,talk...|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
| sci.skeptic,alt....|
| alt.atheism,talk...|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
|         alt.atheism|
+--------------------+
only showing top 20 rows



In [41]:
# le nombre de documents dont la categorie contient rec_baseball
sqlDF3 = spark.sql("SELECT count(s.Newsgroups) as nbr_rec_baseball FROM sport s ")
sqlDF3.show()

+----------------+
|nbr_rec_baseball|
+----------------+
|            1000|
+----------------+



In [42]:
# le nombre de documents dont la categorie contient alt.atheism
sqlDF3 = spark.sql("SELECT count(Newsgroups) as nb_alt_atheism FROM religion")
sqlDF3.show()
 

+--------------+
|nb_alt_atheism|
+--------------+
|          1000|
+--------------+



In [43]:
# Pour prouver qu'il n'y a pas d'intersection entre les deux catégories 
# de plus on a 1000 lignes dans la vue religion et 1000 dans la vue religion
#(en sachant que le nombre total des lignes dans la table document est de 2000)
sqlDF3 = spark.sql("SELECT * FROM religion INTERSECT (SELECT * FROM sport) ")
sqlDF3.show()

+----------+
|Newsgroups|
+----------+
+----------+



**Conclusion:** on a bien deux catégories différentes de documents

### 4.2 Le nombre d'organisations différentes

In [44]:
#le nombre d'organisations différentes
from pyspark.sql.functions import countDistinct
df2=df.select(countDistinct("Organization"))
df2.show()

+----------------------------+
|count(DISTINCT Organization)|
+----------------------------+
|                         485|
+----------------------------+



### 4.3 D’autres statistiques descriptives

Max, Min , Moyenne, écart type par rapport a l'attribut Lines

In [45]:
df.describe("Lines").show()

+-------+-----------------+
|summary|            Lines|
+-------+-----------------+
|  count|             1994|
|   mean|36.48294884653962|
| stddev|48.79137273450094|
|    min|                1|
|    max|               99|
+-------+-----------------+



Le nombre de documents par organization

In [46]:
df.groupBy("Organization").count().show()

+--------------------+-----+
|        Organization|count|
+--------------------+-----+
|           SunSelect|    2|
| Case Western Res...|   39|
| Princeton Univer...|   24|
| University of Ne...|    2|
| S-CUBED, A Divis...|    1|
| Harlequin Ltd. C...|    1|
| University of Ne...|    7|
|   Cured, discharged|    3|
|   Augustana College|    1|
| National Optical...|    2|
| Dept. of Electro...|    1|
| Cabletron System...|    6|
| Welch Medical Li...|    2|
| Red Wolfe @ The ...|    1|
|  Indiana University|   17|
| Johns Hopkins Un...|    7|
| Okcforum Unix Us...|   32|
| Iowa State Unive...|    2|
| Wesleyan University|    7|
| Tektronix, Inc.,...|   11|
+--------------------+-----+
only showing top 20 rows



Le nombre de documents qui ont été créés par chaque auteur

In [47]:
df.groupBy("FROM").count().show()

+--------------------+-----+
|                FROM|count|
+--------------------+-----+
| dpw@sei.cmu.edu ...|    4|
| mccullou@snake2....|    6|
| livesey@solntze....|   70|
| markp@avignon (M...|    1|
| jvigneau@cs.ulow...|    1|
| kilman2y@fiu.edu...|    1|
| darice@yoyo.cc.m...|   14|
| cust_ts@klaava.H...|    2|
| kax@cs.nott.ac.u...|    2|
| acooper@mac.cc.m...|    4|
| "James F. Tims" ...|    1|
| kutluk@ccl.umist...|    1|
| <MVS104@psuvm.ps...|    1|
| davec@silicon.cs...|    1|
| halat@pooh.bears...|   15|
| nancyo@shnext15....|    1|
| David O Hunt <bl...|    1|
| sbradley@scic.in...|    2|
| mikec@sail.LABS....|    2|
| adpeters@sunflow...|    3|
+--------------------+-----+
only showing top 20 rows



##5.A. Transformation de texte : Tout le document
Dans ce qui suit, nous allons créer un nouvel RDD de type pyspark.sql.Row puis un dataframe qui a une seule colonne contenant tout le document. Ceci est pour faire un premier test sur l'algorithme KMeans.


In [48]:
def ToRow_all_document(x):
  entete=x[1][0]
  val=" ".join(entete.values())

  row = Row(document=val+x[1][1])
  return row

all_document=fusion.map(ToRow_all_document)

In [49]:
# create spark dataframe contenant tout le document
all_document_df=spark.createDataFrame(all_document)
all_document_df.printSchema()

root
 |-- document: string (nullable = true)



In [50]:
#Transformation du Texte
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

###5.A.2 Découpage les documents en listes de mots à l’aide de Tokenizer

In [51]:
tokenizer = Tokenizer(inputCol="document", outputCol="words")
#d=df.select("Contenu")
wordsData = tokenizer.transform(all_document_df)
wordsData.show()

+--------------------+--------------------+
|            document|               words|
+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantalo

###5.A.3 Création d'une représentation vectorielle des documents à l’aide de HashingTF

In [107]:
#Création d'une représentation vectorielle des documents à l’aide de HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
featurizedData.show()

+--------------------+--------------------+--------------------+
|            document|               words|            features|
+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[2,4,5,7,8,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, 

##6.A. Groupement des documents ayants des représentations vectorielles proches



Comme nous allons tester l'algorithme de KMeans dans plusieurs cas (document entier/entete et Tf-Idf/normalisation), nous allons créer une fonction K_means qui prend un vecteur et applique l'algorithme de clustering dessus, puis affiche le Silhouette score, le nombre de clusters, leurs centres et la taille de chaque cluster.

In [131]:
# Kmeans  
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import seaborn as sns
import pandas as pd
import matplotlib as plt


def K_means(Datavector):
    # Trains a k-means model.
    kmeans = KMeans().setK(2).setSeed(5)
    model = kmeans.fit(Datavector)
    # make prediction
    predictions = model.transform(Datavector)
    # Evaluate clustering by computing Silhouette score
    evaluator = ClusteringEvaluator()

    # Le Silhouette score est une mesure utilisée pour calculer la qualité technique du clustering , sa valeur est comprise entre 1 et -1
    # 1: un bon clustering
    # 0: Signifie que les clusters sont indifférents, ou nous pouvons dire que la distance entre les clusters n'est pas significative.
    # -1: Signifie que les clusters sont affectés dans le mauvais sens.
    # s=(ba)/max(a,b) ou a est la distance intra-cluster moyenne et b est la distance inter-cluster moyenne

    print("\n\n*********************************************************************************\n\n")
    silhouette = evaluator.evaluate(predictions)
    print("Silhouette with squared euclidean distance = " + str(silhouette))
    print("\n\n*********************************************************************************\n\n")
    # Shows the result.
    centers = model.clusterCenters()
    print("Cluster Centers: ", len(centers))
    for center in centers:
      print(center)

    print("\n\n*********************************************************************************\n\n")
    #Affichage du résultat du clustering
    predictions.groupBy('prediction').count().show()
    
    print("\n\n*********************************************************************************\n\n")


In [114]:
K_means(featurizedData)



**********************************************************


Silhouette with squared euclidean distance = 0.99887352132776


**********************************************************


Cluster Centers:  2
[ 6.12306153  3.85342671  4.4062031   6.21810905  4.62031016  4.75437719
  4.49174587  4.60130065  4.981991    5.92796398  3.51375688  3.96948474
 21.94147074  4.09154577  4.21310655  5.87693847  4.34667334  5.97898949
  3.14507254  3.64482241]
[  83.  111.   55.  110.   84.   78.   83.  123.  116.  117.   75.  145.
 2836.   93.   41.  141.   68.  157.   45.   71.]


**********************************************************


+----------+-----+
|prediction|count|
+----------+-----+
|         1|    1|
|         0| 1999|
+----------+-----+



**********************************************************





  Le Silhouette score est une mesure utilisée pour calculer la qualité technique du clustering , sa valeur est comprise entre 1 et -1 :

     1: un bon clustering
     0: Signifie que les clusters sont indifférents, ou nous pouvons dire
      que la distance entre les clusters n'est pas significative.
     -1: Signifie que les clusters sont affectés dans le mauvais sens.
     s=(ba)/max(a,b) ou a est la distance intra-cluster moyenne et b est la distance inter-cluster moyenne


Notre score obtenu est donc quasi maximal.

Le modèle a trouvé deux clusters bien espacés mais la quasi totalité des prédictions (99,95%) a été assigné au cluster 0.

##7.A. Pour aller plus loin 

###A.1. KMeans avec la pondération Tf-Idf

On renomme la colonne `features` en `features_hache` pour la donner ensuite a la fonction IDF




In [115]:
 featurizedData = featurizedData.selectExpr("document as document","words as words"," features as features_hache")
 featurizedData.show()

+--------------------+--------------------+--------------------+
|            document|               words|      features_hache|
+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[2,4,5,7,8,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, 

#### Pondération des mots avec la formule Tf-*Idf*

In [116]:

idf = IDF(inputCol="features_hache", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.show()

+--------------------+--------------------+--------------------+--------------------+
|            document|               words|      features_hache|            features|
+--------------------+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,

#### KMeans avec ponderation Tf-Idf

In [117]:
#Kmeans pour idf
K_means(rescaledData)



**********************************************************


Silhouette with squared euclidean distance = 0.9623326690517743


**********************************************************


Cluster Centers:  2
[0.52177119 0.42035305 0.08003252 0.4234517  0.30674603 0.09228534
 0.3031003  0.19459049 0.56466917 0.14244486 0.43699812 0.50382377
 0.         0.41363826 0.         0.2196208  0.45930965 0.34303403
 0.48683752 0.46279729]
[4.11124739 3.93626956 0.39830653 3.53307424 2.32339827 0.69523624
 2.4229135  1.63503913 5.47359685 1.11692855 3.46075883 5.02721268
 0.         3.16354209 0.         1.89814007 4.20413497 3.4083718
 4.18614601 3.77357244]


**********************************************************


+----------+-----+
|prediction|count|
+----------+-----+
|         1|   31|
|         0| 1969|
+----------+-----+



**********************************************************




Ici nous avons un Silhouette score qui diminue légèrement baisse, mais nous remarquons que les données sont un peu plus séparées qu'auparavant.

###A.2 Normalisation des vecteurs représentant les documents

#### Normalisation

In [118]:
# normalisation
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
normalizer = Normalizer(inputCol="features_hache", outputCol="features", p=1.0)
l1NormData = normalizer.transform(featurizedData)
l1NormData.show()

+--------------------+--------------------+--------------------+--------------------+
|            document|               words|      features_hache|            features|
+--------------------+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,

#### KMeans avec normalisation

In [119]:
# Kmeans avec normalisation
K_means(l1NormData)



**********************************************************


Silhouette with squared euclidean distance = 0.36088894915766156


**********************************************************


Cluster Centers:  2
[0.05715686 0.03530914 0.04203459 0.05956944 0.04395482 0.0440991
 0.04184161 0.04350036 0.04790209 0.05462928 0.03328339 0.0376798
 0.16346395 0.03881574 0.03894001 0.05650259 0.04165701 0.05594521
 0.02954618 0.03416884]
[0.03716904 0.03033171 0.0488685  0.03864226 0.04151782 0.04578196
 0.03885956 0.04090615 0.03071138 0.0532178  0.02839841 0.02648926
 0.28733672 0.03151724 0.05086917 0.04626162 0.02925748 0.04107195
 0.02368617 0.02910581]


**********************************************************


+----------+-----+
|prediction|count|
+----------+-----+
|         1|  847|
|         0| 1153|
+----------+-----+



**********************************************************




Nous pouvons voir ici que les predictions sont mieux separées mais que le Silhouette score est plus proche de 0, ce qui signigie que la distance entre les clusters n'est pas significative.

##5.B. Transformation de texte : Entête seulement


Comme le résultat du clustering n'a pas donné des résultats satisfaisants, nous allons essayer les meme étapes sur l'entete des documents.

Nous allons commencer par créer un nouvel RDD de type pyspark.sql.Row puis un dataframe qui a une seule colonne ne contenant que l'eentete. Ceci est pour faire un deuxieme test sur l'algorithme KMeans.

In [132]:
def ToRow_entete(x):
  entete=x[1][0]
  val=" ".join(entete.values())
  row = Row(entete=val)
  return row

entete_document_row=fusion.map(ToRow_entete)

In [133]:
Entete_df=spark.createDataFrame(entete_document_row)
Entete_df.printSchema()

root
 |-- entete: string (nullable = true)



###5.B.2 Découpage les documents en listes de mots à l’aide de Tokenizer

In [134]:
tokenizer2 = Tokenizer(inputCol="entete", outputCol="words")

wordsData_entete = tokenizer2.transform(Entete_df)
wordsData_entete.show()

+--------------------+--------------------+
|              entete|               words|
+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|
| alt.atheism  can...|[, alt.atheism, ,...|
| cantaloupe.srv.c...|[, cantalo

###5.B.3 Création d'une représentation vectorielle des documents à l’aide de HashingTF

In [135]:
#Création d'une représentation vectorielle des documents à l’aide de HashingTF
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=20)
featurizedData_entete = hashingTF.transform(wordsData_entete)
featurizedData_entete.show()

+--------------------+--------------------+--------------------+
|              entete|               words|            features|
+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[1,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[2,4,5,7,8,10...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, 

##6.B. Groupement des documents ayants des représentations vectorielles proches

In [136]:
K_means(featurizedData_entete)



*********************************************************************************


Silhouette with squared euclidean distance = 0.23252379882689977


*********************************************************************************


Cluster Centers:  2
[ 1.12418301  1.05954975  1.74001452  1.43718228  1.54901961  1.92302106
  1.37109659  1.54901961  1.13870733  1.47204067  1.02251271  0.9557008
 10.94553377  1.13943355  2.10384895  1.92084241  1.0087146   1.42701525
  0.89251997  0.96877269]
[ 1.3964687   1.47672552  2.19101124  1.21187801  1.77046549  2.31139647
  2.05457464  2.08025682  1.12841091  1.62760835  1.38202247  1.01605136
 14.12199037  1.87800963  2.00160514  1.83627608  1.2600321   1.61637239
  0.97913323  1.28410915]


*********************************************************************************


+----------+-----+
|prediction|count|
+----------+-----+
|         1|  623|
|         0| 1377|
+----------+-----+



***************************************************

Nous pouvons voir ici que les predictions sont plus separées que dans le cas des docu ents entiers. 

Mais le Silhouette score est plus proche de 0, ce qui signigie que la distance entre les clusters n'est pas significative.

##7.B. Pour aller plus loin

###B.1. KMeans avec ponderation Tf-Idf

#### Pondération des mots avec la formule Tf-*Idf*

Renommer la colonne `features` en `features_hache`

In [137]:
 featurizedData_entete = featurizedData_entete.selectExpr("entete as entete","words as words"," features as features_hache")

In [138]:
idf = IDF(inputCol="features_hache", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData_entete = idfModel.transform(featurizedData_entete)
rescaledData_entete.show()

+--------------------+--------------------+--------------------+--------------------+
|              entete|               words|      features_hache|            features|
+--------------------+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[1,2,3,4,5,6,...|(20,[1,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,3,

#### KMeans avec ponderation Tf-Idf

In [139]:
K_means(rescaledData_entete)



*********************************************************************************


Silhouette with squared euclidean distance = 0.2154110917054862


*********************************************************************************


Cluster Centers:  2
[0.11410959 0.14774254 0.03984234 0.10781509 0.1112571  0.04342768
 0.11692905 0.07808274 0.15253256 0.04022586 0.16291172 0.1385629
 0.         0.1540329  0.         0.08400134 0.13586948 0.09841349
 0.40446856 0.14349275]
[0.11303693 0.14354843 0.03483606 0.10122597 0.11932243 0.04333716
 0.11761493 0.08035191 0.14182566 0.03997453 0.1519968  0.13840153
 0.         0.15012429 0.         0.07644409 0.12604553 0.09503113
 0.07872695 0.1509392 ]


*********************************************************************************


+----------+-----+
|prediction|count|
+----------+-----+
|         1| 1513|
|         0|  487|
+----------+-----+



*********************************************************************************




Comme l'entete est courte, et que les mots qui se repetent sont des mots qui correspondent aux catégories, le Tf-Idf ne fonctionne pas tres bien (car il considere que les mots les plus répétés comme non pertinents).

###B.2. KMeans avec  normalisation

####Normalisation

In [140]:
# normalisation
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
normalizer = Normalizer(inputCol="features_hache", outputCol="features", p=1.0)
l1NormData_entete = normalizer.transform(featurizedData_entete)
l1NormData_entete.show()

+--------------------+--------------------+--------------------+--------------------+
|              entete|               words|      features_hache|            features|
+--------------------+--------------------+--------------------+--------------------+
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,5,6,...|(20,[0,1,2,3,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
| alt.atheism  can...|[, alt.atheism, ,...|(20,[1,2,3,4,5,6,...|(20,[1,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,5,6,7,9,...|(20,[1,2,5,6,7,9,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[0,2,3,4,5,6,...|(20,[0,2,3,4,5,6,...|
| cantaloupe.srv.c...|[, cantaloupe.srv...|(20,[1,2,3,

####KMeans avec normalisation

In [141]:
K_means(l1NormData_entete)



*********************************************************************************


Silhouette with squared euclidean distance = 0.14966867843973244


*********************************************************************************


Cluster Centers:  2
[0.02983836 0.03289355 0.04984843 0.03849039 0.04015886 0.05369461
 0.0442604  0.04458531 0.03263492 0.039472   0.03001639 0.02471977
 0.27494399 0.03771288 0.05253869 0.0516627  0.03136576 0.03781053
 0.02456342 0.02878904]
[0.03198619 0.02624152 0.044923   0.03064597 0.04358449 0.05008385
 0.03382938 0.04185756 0.02348745 0.03859332 0.02672645 0.02514087
 0.34780869 0.02961522 0.054876   0.04423896 0.02199324 0.03815162
 0.02190107 0.02431513]


*********************************************************************************


+----------+-----+
|prediction|count|
+----------+-----+
|         1|  896|
|         0| 1104|
+----------+-----+



*********************************************************************************




La séparation des documents semble plus ou moins équilibrées, mais le Silhouette score est quasiment nul.

***Conclusion du clustering:***
Nos résultats de clustering ne sont donc pas tres satisfaisants. 

On pourrait envisager de nettoyer un peu plus nos données ou de prendre en considérations d'autres scores tel que la NMI entre le partitionnement trouvé et les vraies classes et qui peut être un meilleur indicateur. 
